Filtrado de spam en mensajes de texto (SMS)
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/predictive-analytics/blob/master/06-NaiveBayes-IPy-SMS-spam.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/predictive-analytics/blob/master/06-NaiveBayes-IPy-SMS-spam.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/predictive-analytics/blob/master/LICENSE)  
[Readme](https://github.com/jdvelasq/predictive-analytics/blob/master/readme.md)

In [27]:
#
# Lee el archivo. La función readlines() retorna una 
# una lista de strings donde cada string es una linea
# del archivo original.
#
sms_raw = open('data/sms_spam.csv').readlines()
#
# Elimina las comillas dobles al principio y al
# final
#
lines = []
for line in sms_raw:
    lines.append(line[1:-2]) 
sms_raw = lines
#
# Convierte cada linea en una lista de strings, 
# partiendo la línea original por las comas.
# 
sms_raw = [x.split('","') for x in sms_raw]
#
# Elimina la primera fila que corresponde a los
# encabezamientos
#
sms_raw = sms_raw[1:]
#
# Imprime los primeros cinco registros
#
sms_raw[0:5]
#
## str(sms_raw)
##
## 'data.frame':	5574 obs. of  2 variables:
##  $ type: chr  "ham" "ham" "spam" "ham" ...
##  $ text: chr  "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..." "Ok lar... Joking wif u oni..." "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question("| __truncated__ "U dun say so early hor... U c already then say..." ...
##

[['ham',
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'],
 ['ham', 'Ok lar... Joking wif u oni...'],
 ['spam',
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"],
 ['ham', 'U dun say so early hor... U c already then say...'],
 ['ham', "Nah I don't think he goes to usf, he lives around here though"]]

In [28]:
#
# Separa el texto y el tipo de mensaje
# 
sms_raw_type = [x[0] for x in sms_raw]
sms_raw_text = [x[1] for x in sms_raw]

In [ ]:
# Se convierte la columna type, que contiene strings, en un factor.
## sms_raw$type <- factor(sms_raw$type)
## str(sms_raw$type)
##
##  Factor w/ 2 levels "ham","spam": 1 1 2 1 1 2 1 1 2 2 ...
##

In [ ]:
#
# Se crea una función que mimifica 
# la función table de R
#
def table(x):
    return {y:sms_raw_type.count(y) for y in set(x)}

In [32]:
#
# cuenta la cantidad de ham y spam
#
{x:sms_raw_type.count(x) for x in set(sms_raw_type)}

{'ham': 4827, 'spam': 747}

In [ ]:
# Cantidad de casos para cada tipo de mensaje.
## table(sms_raw$type)
##
## 
##  ham spam 
## 4827  747 
##

In [ ]:
# Se convierte el conteo en probabilidades.
## round(prop.table(table(sms_raw$type)) * 100, digits = 1)
##
## 
##  ham spam 
## 86.6 13.4 
##

# Preparación de los datos

In [ ]:
# El paquete tm se usa para minería de texto
# install.packages("tm")
# install.packages("NLP")
# install.packages("SnowballC")
## library(NLP)
## library(tm)
## library(SnowballC)

In [ ]:
# Se crea un corpus que es una colección de documentos.
## sms_corpus <- VCorpus(VectorSource(sms_raw$text))
## print(sms_corpus)
##
## <<VCorpus>>
## Metadata:  corpus specific: 0, document level (indexed): 0
## Content:  documents: 5574
##

In [ ]:
# Se pueden seleccionar elementos particulares dentro del corpus.
## inspect(sms_corpus[1:2])
##
## <<VCorpus>>
## Metadata:  corpus specific: 0, document level (indexed): 0
## Content:  documents: 2
## 
## [[1]]
## <<PlainTextDocument>>
## Metadata:  7
## Content:  chars: 111
## 
## [[2]]
## <<PlainTextDocument>>
## Metadata:  7
## Content:  chars: 29
## 
##

In [ ]:
# Para ver un mensaje de texto en particular se usa as.character
## as.character(sms_corpus[[1]])
##
## [1] "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
##

In [ ]:
# Para ver varios documentos se usa lapply
## lapply(sms_corpus[1:2], as.character)
##
## $`1`
## [1] "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
## 
## $`2`
## [1] "Ok lar... Joking wif u oni..."
##

In [ ]:
# Se convierten todas las letras en minúsculas
## sms_corpus_clean <- tm_map(sms_corpus, 
##                            content_transformer(tolower))

# Se eliminan los dígitos
## sms_corpus_clean <- tm_map(sms_corpus_clean, 
##                            removeNumbers)

# Se eliminan las stop-words
## sms_corpus_clean <- tm_map(sms_corpus_clean,
##                            removeWords, # funcion que remueve las palabras
##                            stopwords()) # lista de palabras a remover

# Se remueve la puntuación.
## sms_corpus_clean <- tm_map(sms_corpus_clean, 
##                            removePunctuation)

# Se transforman a infinitivo las conjugaciones.
## sms_corpus_clean <- tm_map(sms_corpus_clean, 
##                            stemDocument)

# Se remueven espacios en blanco adicionales.
## sms_corpus_clean <- tm_map(sms_corpus_clean, 
##                            stripWhitespace)

In [ ]:
# Para ver el efecto de las transformaciones
# realizadas, a continuación se muestran los 
# mensajes originales y los transformados.
# Mensajes antes de realizar la limpieza.
## lapply(sms_corpus[1:3], as.character)
##
## $`1`
## [1] "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
## 
## $`2`
## [1] "Ok lar... Joking wif u oni..."
## 
## $`3`
## [1] "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
##

In [ ]:
# Mensajes despues de limpiar.
## lapply(sms_corpus_clean[1:3], as.character)
##
## $`1`
## [1] "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"
## 
## $`2`
## [1] "ok lar joke wif u oni"
## 
## $`3`
## [1] "free entri wkli comp win fa cup final tkts st may text fa receiv entri questionstd txt ratetc appli s"
##

In [ ]:
# Se crea la matriz de términos del documento
## sms_dtm <- DocumentTermMatrix(sms_corpus_clean)

In [ ]:
# Es posible crear la matriz de términos del documento
# sin pasar por el preprocesmiento previo y realizarlo
# directamente en la llamada a la función
## sms_dtm2 <- 
## DocumentTermMatrix(sms_corpus, 
##                    control = list(tolower = TRUE, 
##                                   removeNumbers = TRUE,
##                                   stopwords = TRUE, 
##                                   removePunctuation = TRUE,
##                                   stemming = TRUE))

In [ ]:
## sms_dtm
##
## <<DocumentTermMatrix (documents: 5574, terms: 6592)>>
## Non-/sparse entries: 42608/36701200
## Sparsity           : 100%
## Maximal term length: 40
## Weighting          : term frequency (tf)
##

In [ ]:
## sms_dtm2
##
## <<DocumentTermMatrix (documents: 5574, terms: 6995)>>
## Non-/sparse entries: 43713/38946417
## Sparsity           : 100%
## Maximal term length: 40
## Weighting          : term frequency (tf)
##

In [ ]:
# Creación de los conjuntos de entrenamiento y prueba.
## sms_dtm_train <- sms_dtm[1:4169, ]
## sms_dtm_test  <- sms_dtm[4170:5559, ]
## sms_train_labels <- sms_raw[1:4169, ]$type
## sms_test_labels  <- sms_raw[4170:5559, ]$type

In [ ]:
# Distribución de los datos en el conjunto de entrenamiento.
## prop.table(table(sms_train_labels))
##
## sms_train_labels
##       ham      spam 
## 0.8647158 0.1352842 
##

In [ ]:
# Distribución de los datos en el conjunto de prueba.
## prop.table(table(sms_test_labels))
##
## sms_test_labels
##       ham      spam 
## 0.8697842 0.1302158 
##

In [ ]:
# En este caso es más útil visualizar 
# una nube que indique la frecuencia de las
# palabras
# install.packages("wordcloud")
# install.packages("RColorBrewer")
## library(RColorBrewer)
## library(wordcloud)
## wordcloud(sms_corpus_clean, 
##           min.freq = 50,         # número de veces que debe aparecer una palabra
##           random.order = FALSE)  # palabras más frecuentes en el centro
##
## plot without title
##

In [ ]:
# Palabras que más aparecen en mensajes spam y válidos.
## spam <- subset(sms_raw, type == "spam")
## ham <- subset(sms_raw, type == "ham")
## wordcloud(spam$text, max.words = 40, scale = c(3, 0.5))
## wordcloud(ham$text, max.words = 40, scale = c(3, 0.5))
##
## plot without title
##

In [ ]:
# Palabras que aparecen al menos en 5 mensajes.
## sms_freq_words <- findFreqTerms(sms_dtm_train, 5)
## str(sms_freq_words)
##
##  chr [1:1157] "£wk" "abiola" "abl" "abt" "accept" "access" "account" ...
##

In [ ]:
# Se construyen conjuntos de entrenamiento y prueba 
# con las palabras que aparecen al menos en 5 mensajes.
## sms_dtm_freq_train<- sms_dtm_train[ , sms_freq_words]
## sms_dtm_freq_test <- sms_dtm_test[ , sms_freq_words]

In [ ]:
# Se convierte la frecuencia de ocurrencia a "Yes" y "No"
## convert_counts <- 
## function(x) {
##     x <- ifelse(x > 0, "Yes", "No")
## }
#
## sms_train <- apply(sms_dtm_freq_train, 
##                    MARGIN = 2,
##                    convert_counts)
#
## sms_test <- apply(sms_dtm_freq_test, 
##                   MARGIN = 2,
##                   convert_counts)

# Entrenamiento del modelo

In [ ]:
# Carga la librería
# install.packages("e1071")
## library(e1071)
## sms_classifier <- naiveBayes(sms_train, 
##                              sms_train_labels)

# Evaluación del modelo

In [ ]:
# Se pronostica para los datos de prueba.
## sms_test_pred <- predict(sms_classifier, sms_test)
## head(sms_test_pred)
##
## [1] spam ham  ham  ham  ham  ham 
## Levels: ham spam
##

In [ ]:
# Se calcula la matriz de confusión.
## table(sms_test_labels, sms_test_pred)
##
##                sms_test_pred
## sms_test_labels  ham spam
##            ham  1200    9
##            spam   20  161
##

In [ ]:
# Se calcula la probabilidad de que cada mensaje sea 
# válido o spam para el conjunto de prueba. 
# Los resultados muestran que los mensajes
# son clasificados correctamente.
## sms_test_prob <- predict(sms_classifier, sms_test, type = "raw")
## head(sms_test_prob)
##
##      ham          spam        
## [1,] 1.142967e-14 1.000000e+00
## [2,] 9.963283e-01 3.671735e-03
## [3,] 9.999824e-01 1.764894e-05
## [4,] 1.000000e+00 8.310837e-09
## [5,] 9.999316e-01 6.839232e-05
## [6,] 9.999987e-01 1.301622e-06
##

In [ ]:
# Resulta más conveniente preparar una nueva tabla que
# muestre la clasificación y no únicamente las 
# probabilidades.
## sms_results <- data.frame(actual_type = sms_test_labels,
##                           predict_type = sms_test_pred,
##                           prob_ham = sms_test_prob[,1],
##                           prob_spam = sms_test_prob[,2])
## head(sms_results)
##
##   actual_type predict_type prob_ham     prob_spam   
## 1 spam        spam         1.142967e-14 1.000000e+00
## 2 ham         ham          9.963283e-01 3.671735e-03
## 3 ham         ham          9.999824e-01 1.764894e-05
## 4 ham         ham          1.000000e+00 8.310837e-09
## 5 ham         ham          9.999316e-01 6.839232e-05
## 6 ham         ham          9.999987e-01 1.301622e-06
##

In [ ]:
# install.packages("gmodels")
## library(gmodels)
## CrossTable(sms_test_pred, 
##            sms_test_labels,
##            prop.chisq = FALSE, 
##            prop.t = FALSE,
##            dnn = c('predicted', 'actual'))
##
## 
##  
##    Cell Contents
## |-------------------------|
## |                       N |
## |           N / Row Total |
## |           N / Col Total |
## |-------------------------|
## 
##  
## Total Observations in Table:  1390 
## 
##  
##              | actual 
##    predicted |       ham |      spam | Row Total | 
## -------------|-----------|-----------|-----------|
##          ham |      1200 |        20 |      1220 | 
##              |     0.984 |     0.016 |     0.878 | 
##              |     0.993 |     0.110 |           | 
## -------------|-----------|-----------|-----------|
##         spam |         9 |       161 |       170 | 
##              |     0.053 |     0.947 |     0.122 | 
##              |     0.007 |     0.890 |           | 
## -------------|-----------|-----------|-----------|
## Column Total |      1209 |       181 |      1390 | 
##              |     0.870 |     0.130 |           | 
## -------------|-----------|-----------|-----------|
## 
##  
##

In [ ]:
# Mensajes con clasificación errónea.
# Resulta muy importante determinar porque los 
# mensajes están mal clasificados
## head(subset(sms_results, actual_type != predict_type))
##
##     actual_type predict_type prob_ham    prob_spam  
## 45  spam        ham          0.988193712 0.011806288
## 54  ham         spam         0.003955372 0.996044628
## 85  ham         spam         0.277074268 0.722925732
## 88  spam        ham          0.988551588 0.011448412
## 130 spam        ham          0.993500148 0.006499852
## 205 spam        ham          0.895915228 0.104084772
##

In [ ]:
# Sin embargo, es mucho más intersante extraer
# mensajes con probabilidades numéricamente 
# cercanas a 0.5. Estos podrían generar ambiguedad
# en la clasificación.
## head(subset(sms_results, prob_spam > 0.40 & prob_spam < 0.60))
##
##     actual_type predict_type prob_ham  prob_spam
## 104 spam        spam         0.4885929 0.5114071
## 320 ham         spam         0.4702446 0.5297554
## 694 ham         spam         0.4916055 0.5083945
## 709 spam        spam         0.4885929 0.5114071
## 817 spam        spam         0.4885929 0.5114071
## 873 spam        spam         0.4885929 0.5114071
##

In [ ]:
# Mensajes mal clasificados con probabilidad cercana a 0.5 
## head(subset(sms_results, prob_spam > 0.40 & prob_spam < 0.60 & actual_type != predict_type))
##
##     actual_type predict_type prob_ham  prob_spam
## 320 ham         spam         0.4702446 0.5297554
## 694 ham         spam         0.4916055 0.5083945
##

**Ejercicio.--** El código presentado a continuación genera una curva ROC para este clasificador. Cómo se interpreta?

In [ ]:
# install.packages("ROCR")
## library(ROCR)
## pred <- prediction(predictions = sms_results$prob_spam,
##                    labels = sms_results$actual_type)
## perf <- performance(pred, measure = "tpr", x.measure = "fpr")
## plot(perf, 
##      main = "ROC curve for SMS spam filter",
##      col = "blue", 
##      lwd = 3)
## abline(a = 0, b = 1, lwd = 2, lty = 2)
##
## Loading required package: gplots
## 
## Attaching package: ‘gplots’
## 
## The following object is masked from ‘package:wordcloud’:
## 
##     textplot
## 
## The following object is masked from ‘package:stats’:
## 
##     lowess
## 
##

In [ ]:
# Se computa el área bajo la curva.
## perf.auc <- performance(pred, measure = "auc")
## str(perf.auc)
## unlist(perf.auc@y.values)
##
## Formal class 'performance' [package "ROCR"] with 6 slots
##   ..@ x.name      : chr "None"
##   ..@ y.name      : chr "Area under the ROC curve"
##   ..@ alpha.name  : chr "none"
##   ..@ x.values    : list()
##   ..@ y.values    :List of 1
##   .. ..$ : num 0.995
##   ..@ alpha.values: list()
##

---

Filtrado de spam en mensajes de texto (SMS)
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/predictive-analytics/blob/master/06-NaiveBayes-IPy-SMS-spam.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/predictive-analytics/blob/master/06-NaiveBayes-IPy-SMS-spam.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/predictive-analytics/blob/master/LICENSE)  
[Readme](https://github.com/jdvelasq/predictive-analytics/blob/master/readme.md)